In [ ]:
# Complete tinyGPT
# Then build and train gpt2
# Papers - Attention is all you need
# GPT 2/3 paper

# Llamac/llama2.c
# Cuda 

#### commands
""""
To see the gpu stats
nvidia-smi 
# To see al the cpu stats


"""

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("Supports BF16:", torch.cuda.is_bf16_supported())
print("Supports TF32:", torch.backends.cuda.matmul.allow_tf32)
print("Supports FP16:", torch.cuda.get_device_capability(0)[0] >= 7)

if torch.cuda.is_available():
    gpu_id = 0  # Change if you have multiple GPUs
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
    print(f"Total GPU memory: {total_memory / (1024**3):.2f} GB")
    # print(f"Allocated memory: {torch.cuda.memory_allocated(gpu_id) / (1024**2):.2f} MB")
    # print(f"Reserved memory : {torch.cuda.memory_reserved(gpu_id) / (1024**2):.2f} MB")
else:
    print("CUDA not available")

CUDA available: True
GPU name: NVIDIA GeForce GTX 1650
Supports BF16: True
Supports TF32: False
Supports FP16: True
Total GPU memory: 4.00 GB
Allocated memory: 0.00 MB
Reserved memory : 2.00 MB


In [ ]:
# Calculation of Time and money needed to run the model
# Calculate the number of parmameters that model contains


In [ ]:
# import torch
# how many neurons and number of values needed to represent that neuron
# GPT-> Token embedding, position embedding, Module List, lmhead
# Module List -> n_layer number of blocks
# Block -> Casual self attention + Feed forward network
# FeedForward network -> 2 linear model with gelu in between
# Self attention block -> casual attention + projection
vocab_size = 50304 
block_size = 1024
n_embd = 768
n_layer = 12
n_head = 12
batch_size = 12



wte: 38633472, wpe: 786432
total parameters: 124474370
non decay parameters: 119808
non decay parameters: 9984
decay parameters: 124354562


In [25]:
# calculate in a single loop run how much memory is required
model_size = (total_parameters * 2) / (2 ** 20)
# Size of the model if it is stored in FP16
print(f"{model_size:.2f} MB")

237.42 MB


In [31]:
input_size = (batch_size * block_size * n_embd) * 2
# for activation input size becomes
input_size = input_size * 3
model_weight_size = total_parameters * 2
# optimizer (AdamW) stores 3 copies(weights + momentum + velocity)
optimizer_size = model_weight_size * 3

total_memory = input_size + model_weight_size + optimizer_size
print(f"total size: {total_memory//(2**20):.2f}")

total size: 1003.00


In [1]:
# model config
vocab_size = 50304 
block_size = 1024
n_embd = 768
n_layer = 12
n_head = 12
batch_size = 12

In [2]:
# with Bias calculation
def params():
    qkv_proj_param = (n_embd * 3 * n_embd) + (3 * n_embd)
    out_proj_param = (n_embd * n_embd) + (n_embd)
    attn_param = qkv_proj_param + out_proj_param

    feed_ln1_param = (n_embd * 4 * n_embd) + (4 * n_embd)
    feed_ln2_param = (4 * n_embd * n_embd) + n_embd
    feed_param = feed_ln1_param + feed_ln2_param

    layer_norm_param = 2 * 2 * n_embd
    block_param =  layer_norm_param + attn_param + feed_param # a layer norm contains 2 param
    wte = vocab_size * n_embd
    wpe = block_size * n_embd
    # decay_param = (3 * n_embd) + (n_embd) + (4 * n_embd) + n_embd + 4 * n_embd
    non_decay_param = n_layer * (13 * n_embd)
    total_parameters = (wte + wpe + 2 + n_layer * block_param)
    print(f"wte: {wte}, wpe: {wpe}")
    print(f"total parameters: {total_parameters}")
    print(f"non decay parameters: {non_decay_param}")
    print(f"non decay parameters: {non_decay_param//n_layer}")
    print(f"decay parameters: {total_parameters - non_decay_param}")
    # find out decayed and non decay params of the model

params()

wte: 38633472, wpe: 786432
total parameters: 124474370
non decay parameters: 119808
non decay parameters: 9984
decay parameters: 124354562


In [ ]:
# model saved in FP16 - 2 bytes
